In [1]:
import pyaccel
import pymodels
import numpy as np
import matplotlib.pyplot as plt
from mathphys.functions import save_pickle, load_pickle

import touschek_pack.functions as tousfunc
import touschek_pack.Classes as tousclass
import pyaccel.optics as pyop

In [2]:
fitm = pymodels.si.create_accelerator()
fitm = pymodels.si.fitted_models.vertical_dispersion_and_coupling(fitm)
eqparams = pyaccel.optics.beam_envelope.EqParamsFromBeamEnvelope(fitm)

# parameters to generate the beam
h_emitt = eqparams.emit1
v_emitt = h_emitt/100
sigmae = eqparams.espread0
bun_len = eqparams.bunlen

# defining optcis
twiss,*_ = pyop.calc_twiss(fitm, indices='closed')

# defining the index 
dicty = pyaccel.lattice.find_dict(fitm, 'fam_name')
nlk_index = dicty['InjNLKckr'][0]
spos = pyaccel.lattice.find_spos(fitm, indices='closed')

# generating gaussian beam.
bunch = pyaccel.tracking.generate_bunch(5000, envelope=None, emit1=h_emitt,
                                emit2=v_emitt, sigmae= sigmae, sigmas=bun_len, optics=twiss[nlk_index])


In [3]:
def track_mchn_stdy(acc, bunch, param, increment, index, nturn):

    if 'x' in param:
        bunch[0] += increment
    elif 'y' in param:
        bunch[2] += increment
    elif 'xl' in param:
        bunch[1] += increment
    elif 'yl' in param:
        bunch[3] += increment
    
    tracked = pyaccel.tracking.ring_pass(acc, particles=bunch,
                                         nr_turns=nturn, turn_by_turn= True,
                                         element_offset= index, parallel=True)
    
    part_out, lost_flag, turn_lost, index_lost, plane_lost = tracked

    turnl_element = []

    for i,iten in enumerate(turn_lost):
        if iten == nturn and index_lost[i] == index: # ignora elétrons que não foram perdidos
            pass
        else:
            turnl_element.append((iten, index_lost[i]))
    
    return turnl_element

# ao passo que a função acima faz o tracking para um bunch a função abaixo varia automaticamente os parametros para obtermos um resultado mais automatizado e rápido

def varying_incmnts(acc,bunch,par,rng, index, nturn):
    trackf_lostpos = []
    inc = np.linspace(0, 1e-1, rng) 
    for iten in inc:
        
        trackf_lostpos.append(track_mchn_stdy(acc, bunch, par, increment=iten, index=index, nturn=nturn))

    return trackf_lostpos



In [4]:
# verificando se houve uma redução das particulas que foram perdidas com a introdução da variação da altura da camara de vacuo

tousan = tousclass.Tous_analysis(fitm)
nturn = tousan.nturns
probable_pickle = varying_incmnts(fitm, bunch, 'x', 50, nlk_index, nturn)


In [ ]:
# # anotações que eu pensei que pudessem ser importantes

# # definindo também o modelo fitado para calcular a distribuição de equilíbrio do feixe
# # eu não sei exatamente se essa celula no meu código é util, porque em um primeiro momento vamos variar apenas
# # as posições x, y, x' e y'

# fit_acc = pymodels.si.create_accelerator()
# fit_acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(fit_acc)
# tousan = tousclass.Tous_analysis(fit_acc)
# deltas = tousan.deltas
# n_turns = tousan.nturns

# orb = pyaccel.tracking.find_orbit6(acc, indices=[0, nlk_index])
# orb = orb[:, 1]
# np.zeros((6, deltas.size)), orb[:, None]

# rin = np.zeros((6, deltas.size))
# rin += orb[:,None]

# # caso em algum momento seja necessário gerar uma distribuição de feixe gaussiano,
# # com algumas linhas de código é possível fazer esta tarefa
# # eu poderia fazer assim também, mas como essa abordagem é mais complicada porque eu teria que propagar a matriz do booster veremos como fazer assim depois

# mean = np.zeros(6)
# env = pyop.beam_envelope.calc_beamenvelope(fit_acc)
# cov = env[nlk_index]
# particles = np.random.multivariate_normal(mean, cov, size=1).T

# # célula colocada aqui simplesmente para teste de verificação
# # ao executar o código abaixo é possível observar a distribuição gaussiana das posições em x
# plt.figure()
# plt.hist(particles[0], bins=200)
# plt.show()

# # esse procedimento é feito para ir variando os parâmetros de posição 
# vecy = np.linspace(4.8332669321064365e-09, 4.8332669321064365e-02, 1000)
# vecx = np.linspace(2.25702434e-08,2.25702434e-02, 1000)
# # com a função vstack do numpy podemos juntar dois arrays e formar uma matriz
# np.vstack((vecx, vecy))